<a href="https://colab.research.google.com/github/pbeens/python/blob/master/OECTA_AQ_ABQ_Courses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program retrieves all the AQ and ABQ courses from the OECTA website and stores them in an HTML file.

In [ ]:
# imports
from bs4 import BeautifulSoup
import urllib.request

In [ ]:
# global variable(s)
url = 'https://www.catholicteachers.ca/For-Your-Career/AQ-Courses'
term = '2021 Fall'

In [ ]:
# grab each URL
subject_pages = []
subject_urls_dict = {}

html_page = urllib.request.urlopen(url)
soup = BeautifulSoup(html_page, "html.parser")

# find each subject and link
for link in soup.findAll('a', href=True):
    if str(link).find('Course-Descriptions') > 0:
        subject_pages.append('https://www.catholicteachers.ca' + link['href'])

# process each subject page to find H1 (subject name)
for link in subject_pages:

    html_page = urllib.request.urlopen(link)
    soup = BeautifulSoup(html_page, "html.parser")

    subject = soup.find_all('h1')[1]

    if not subject.text == 'Course Descriptions':
        # add to dict
        subject_urls_dict[subject.text] = link

# check the subjects dictionary
for (subject, url) in subject_urls_dict.items():
    print(f'{subject}: {url}')

print('Done.')

In [ ]:
# find each section (part 1, 2, etc) for each subject
courses_with_sections_dict = {}

for (subject, url) in subject_urls_dict.items():

    html_page = urllib.request.urlopen(url)
    soup = BeautifulSoup(html_page, "html.parser")
    
    sections = soup.find_all('h4')
  
    for section in sections:
        if (subject.find('Autism') > 0) or (subject.find('Grades 7 and 8') > 0):
            course_with_section = subject
        elif section.text == 'Share this page:':
            continue
        elif section.text == 'Single Session':
            course_with_section = subject
        else: # part 1, part 2, specialist
            course_with_section = subject + ' ' + section.text
        # add to dict
        courses_with_sections_dict[course_with_section] = url

# check the courses dictionary
for (k,v) in courses_with_sections_dict.items():
    print(f'{k}: {v}')   

In [ ]:
# create html file with course listings
file = './courses.html'
with open(file, 'w') as f:
  s = f'<HTML>\n<HEAD>\n\t<TITLE>{term}</TITLE>\n</HEAD>\n<BODY>\n'
  f.write(s)
  for course, url in courses_with_sections_dict.items():
    f.write(f'\t<a href="{url}">{course}</a><br>\n')
  s = '</BODY>\n<HTML>'
  f.write(s)
f.close()
print(f'{file} created.')